<a href="https://colab.research.google.com/github/sohomghosh/Financial_Narrative_Summarization_2022/blob/main/FNS2022_scoring_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers

  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.11.0 requires huggingface-hub<0.1.0, but you have huggingface-hub 0.4.0 which is incompatible.


In [ ]:
import transformers
transformers.__version__

'4.17.0'

In [ ]:
!git clone https://github.com/orzhan/t5-long-extract

Cloning into 't5-long-extract'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 5), reused 13 (delta 3), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
!mv t5-long-extract/* ./

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!cd transformers && git reset --hard 143738214 

Cloning into 'transformers'...
remote: Enumerating objects: 107862, done.
remote: Total 107862 (delta 0), reused 0 (delta 0), pack-reused 107862
Receiving objects: 100% (107862/107862), 94.31 MiB | 25.24 MiB/s, done.
Resolving deltas: 100% (78580/78580), done.
HEAD is now at 143738214 Fix the loss calculation of ProphetNet (#13132)


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 76 kB 2.9 MB/s 
     |████████████████████████████████| 264 kB 11.3 MB/s 
     |████████████████████████████████| 56 kB 5.4 MB/s 
     |████████████████████████████████| 212 kB 47.0 MB/s 
     |████████████████████████████████| 134 kB 47.7 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.63.0
    Uninstalling tqdm-4.63.0:
      Successfully uninstalled tqdm-4.63.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.4.0
    Uninstalling huggingface-hub-0.4.0:
      Successfully uninstalled huggingface-hub-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.17.0 requires huggingface-hub<1.0,>=0.1.0, but you have huggingface-hub 0.0.19 which is incompatible.


In [ ]:
#upload english_test
!tar xf testing.tar.gz

In [ ]:
import os
from tqdm import tqdm
import pandas as pd

summaries=[]
refs=[]
ids = [x.replace('.txt','') for x in os.listdir( 'testing/annual_reports' )]
data=[]
data_ids=[]
data_long=[]

# Convert data to csv
for id in tqdm(ids[:]):
  with open('testing/annual_reports/%s.txt' % id) as fin:
    content = fin.read().replace('\x00','').strip()
    data.append([content[:22000], 'X'])
    data_long.append(content)
    data_ids.append(id)
    
data = pd.DataFrame(data)
data.columns=['text','summary']
data.to_csv('tet.csv',index=False)

os.system('''mkdir verification''')
os.system('''mkdir verification/system''')

100%|██████████| 500/500 [00:01<00:00, 452.53it/s]


256

In [ ]:
!python transformers/examples/pytorch/summarization/run_summarization.py \
    --model_name_or_path orzhan/t5-long-extract \
    --source_prefix "sum: " \
    --do_predict \
    --validation_file tet.csv \
    --test_file tet.csv \
    --output_dir /content/tst-summarization \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate \
    --max_source_length 4096 \
    --max_target_length 64

03/30/2022 10:54:58 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/30/2022 10:54:58 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token

In [ ]:
ids = [x.replace('.txt','') for x in os.listdir( 'testing/annual_reports' )]

# Create summaries based on predicted sequences
with open('tst-summarization/generated_predictions.txt','r') as fin:
  preds=fin.readlines()
summaries=[]
aa=[]
trues=[]

for i, row in tqdm(data.iterrows(), total=len(data)):
  pos=None
  # Split text into words
  tt=data_long[i]
  tt_words=['']
  tt_spaces=[]
  prev='a'
  for k in range(0,len(tt)):
    if tt[k].isspace() and prev.isspace():
      tt_spaces[-1] += tt[k]
    elif tt[k].isspace() and not prev.isspace():
      tt_spaces.append(tt[k])
    elif not tt[k].isspace() and prev.isspace():
      tt_words.append(tt[k])
    else:
      tt_words[-1] += tt[k]
    prev = tt[k]
  tt_spaces.append('')
  # Locate prediction in the text (best match)
  pp=preds[i].split()
  posmax=len(pp)
  for p in range(min(len(tt_words)-len(pp), 4000)):
    fail=0
    for j in range(len(pp)):
      if tt_words[p+j] != pp[j]:
        fail+=1         # count different words
    if fail<posmax:     # found new best match
      pos=p
      posmax=fail
      if fail==0:
        break
  if pos is None:
    pos=0
  aa.append(pos)
  predicted=''
  pred_length=1000
  for j in range(pos, pos+pred_length):             # take 1000 words from pos
    predicted += tt_words[j] + tt_spaces[j]
    if j == len(tt_words)-1:
      break
  summaries.append(predicted)
  with open('verification/system/%s_%s.txt' % (data_ids[i],'LIPI'),'w') as fout:
    fout.write(predicted)

100%|██████████| 500/500 [02:43<00:00,  3.05it/s]


In [ ]:
!zip -r /content/english_predictions.zip /content/verification/system

  adding: content/verification/system/ (stored 0%)
  adding: content/verification/system/28425_LIPI.txt (deflated 57%)
  adding: content/verification/system/25594_LIPI.txt (deflated 55%)
  adding: content/verification/system/28433_LIPI.txt (deflated 56%)
  adding: content/verification/system/27140_LIPI.txt (deflated 57%)
  adding: content/verification/system/26466_LIPI.txt (deflated 56%)
  adding: content/verification/system/26012_LIPI.txt (deflated 54%)
  adding: content/verification/system/25696_LIPI.txt (deflated 53%)
  adding: content/verification/system/30665_LIPI.txt (deflated 55%)
  adding: content/verification/system/27525_LIPI.txt (deflated 58%)
  adding: content/verification/system/26171_LIPI.txt (deflated 53%)
  adding: content/verification/system/27131_LIPI.txt (deflated 55%)
  adding: content/verification/system/28897_LIPI.txt (deflated 54%)
  adding: content/verification/system/27671_LIPI.txt (deflated 56%)
  adding: content/verification/system/25281_LIPI.txt (deflated 52